In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE146964"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE146964"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE146964.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE146964.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE146964.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE146964_family.soft.gz', 'GSE146964_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Ovarian_Cancer/GSE146964/GSE146964_family.soft.gz
Matrix file: ../../input/GEO/Ovarian_Cancer/GSE146964/GSE146964_series_matrix.txt.gz


Background Information:
!Series_title	"Unraveling Tumor-Immune Heterogeneity in Advanced Ovarian Cancer Uncovers Immunogenic Effect of Chemotherapy [naïve]"
!Series_summary	"In metastatic cancer, the degree of heterogeneity of the tumor-immune microenvironment and its molecular underpinnings remain largely unstudied. To characterize the tumor-immune interface at baseline and during neoadjuvant chemotherapy in high-grade serous ovarian cancer (HGSOC), we performed immunogenomics analysis of treatment-naive and paired pre/post-chemotherapy treated samples. In treatment-naive HGSOC, we find that immune cell-excluded and inflammatory microenvironments co-exist within the same individuals and within the same tumor sites, indicating ubiquitous variability in immune cell infiltration. Analysis of tumor microenvironment cell composition, DNA copy number, mutations and gene expression showed that immune cell exclusion was associated with amplification of Myc target genes and increased expressio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Based on the previously provided Sample Characteristics Dictionary:
# {0: ['subject id: 10', 'subject id: 06', 'subject id: 01', 'subject id: 04', 'subject id: 05', 'subject id: 13', 'subject id: 16', 'subject id: 17'], 
#  1: ['gender: Female'], 
#  2: ['tissue: Omentum', 'tissue: Paracolic gutter', 'tissue: Ovary', 'tissue: Adnexal surface', 'tissue: Pouch of Douglas', 'tissue: Right psoas', 'tissue: Serosa'], 
#  3: ['condition: Malignant'], 
#  4: ['tumor type: High grade serous ovarian cancer'], 
#  5: ['tumor stage: IIIC', 'tumor stage: IV', 'tumor stage: IVB'], 
#  6: ['sample type: FFPE']}

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains mRNA microarrays,
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Ovarian Cancer): 
# Looking at the sample characteristics dictionary, all samples have "condition: Malignant"
# and "tumor type: High grade serous ovarian cancer" (rows 3 and 4)
# Since all samples are cancer cases with no controls for comparison, 
# there's no variability in the trait value, making it unusable for associative studies
trait_row = None

# For age:
# There is no age information in the sample characteristics dictionary
age_row = None

# For gender:
# All samples are from females (row 1)
# Since there's only one value (all female), there's no variability in gender
gender_row = None

# 2.2 Data Type Conversion Functions

# Define conversion functions (even though we won't use them due to data unavailability)
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'malignant' in value.lower() or 'cancer' in value.lower():
        return 1
    elif 'normal' in value.lower() or 'healthy' in value.lower() or 'benign' in value.lower():
        return 0
    return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if pd.isna(value) or value is None:
        return None
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (no useful clinical data available for our analysis)
# We determined this dataset doesn't have the necessary trait variability for our study


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 138745
First 20 gene/probe identifiers:
Index(['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st',
       'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st', 'AFFX-BkGr-GC08_st',
       'AFFX-BkGr-GC09_st', 'AFFX-BkGr-GC10_st', 'AFFX-BkGr-GC11_st',
       'AFFX-BkGr-GC12_st', 'AFFX-BkGr-GC13_st', 'AFFX-BkGr-GC14_st',
       'AFFX-BkGr-GC15_st', 'AFFX-BkGr-GC16_st', 'AFFX-BkGr-GC17_st',
       'AFFX-BkGr-GC18_st', 'AFFX-BkGr-GC19_st', 'AFFX-BkGr-GC20_st',
       'AFFX-BkGr-GC21_st', 'AFFX-BkGr-GC22_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers, starting with "AFFX-", appear to be from Affymetrix microarray probes
# They're not standard human gene symbols and will need to be mapped to gene symbols
# The "AFFX-" prefix is typically used for Affymetrix control probes, and the "_st" suffix
# indicates these are likely from a newer generation Affymetrix array

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1', 'TC0100006435.hg.1', 'TC0100006436.hg.1'], 'probeset_id': ['TC0100006432.hg.1', 'TC0100006433.hg.1', 'TC0100006434.hg.1', 'TC0100006435.hg.1', 'TC0100006436.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '28046', '29554', '52473', '62948'], 'stop': ['14412', '29178', '31109', '53312', '63887'], 'total_probes': [10.0, 6.0, 10.0, 10.0, 10.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000002844 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// OTTHUMT00000362751 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'spopoybu.aAug10-unspliced // spopoybu // Transcript Identified by AceView // --- // ---', 'NR_036267 // MIR1302-10 // microRNA 1302-10 // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene identifiers and gene annotation data:
# Gene expression data uses Affymetrix IDs like "AFFX-BkGr-GC03_st"
# The gene annotation data uses "ID" or "probeset_id" for identifiers
# For gene symbols, we need to extract them from the "gene_assignment" column

# 2. Extract mapping between probe IDs and gene symbols
# Create mapping dataframe with relevant columns
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
# Rename 'gene_assignment' to 'Gene' to match the expected column name in apply_gene_mapping function
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# First make sure the gene expression data is loaded
if 'gene_data' not in locals():
    gene_data = get_genetic_data(matrix_file)
    
# Apply the mapping function to convert probes to genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print information about the resulting gene expression data
print(f"Mapped gene expression data shape: {gene_data.shape}")
print(f"First few gene symbols: {gene_data.index[:5].tolist()}")
print(f"Number of columns (samples): {len(gene_data.columns)}")


Mapped gene expression data shape: (34003, 38)
First few gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']
Number of columns (samples): 38


### Step 7: Data Normalization and Linking

In [8]:
# 1. Since we already normalized gene symbols in the previous step (6),
# we can use the gene_data that was already processed

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Since we determined in step 2 that there's no usable trait variation 
# (all samples are cancer cases with no controls), we'll create a clinical dataframe
# but note this limitation

# Create a clinical dataframe with the trait (Ovarian_Cancer)
sample_ids = gene_data.columns.tolist()
print(f"Sample IDs from gene data: {len(sample_ids)} samples")

# Create clinical dataframe, but note that all samples have the same trait value
clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
clinical_df.loc[trait] = 1  # All samples are ovarian cancer tumors

print(f"Clinical data shape: {clinical_df.shape}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
print("Handling missing values...")
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
print("Checking for bias in features...")
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains expression data for ovarian cancer tumors. All samples are tumor samples with no controls, so trait bias is expected and the dataset is not suitable for case-control analysis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True, 
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable (though we expect it won't be due to trait bias)
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to trait bias. Final linked data not saved.")

Gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE146964.csv
Sample IDs from gene data: 38 samples
Clinical data shape: (1, 38)
Clinical data saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE146964.csv
Shape of linked data: (38, 34004)
Handling missing values...


/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Shape of linked data after handling missing values: (38, 34004)
Checking for bias in features...
Quartiles for 'Ovarian_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Ovarian_Cancer' in this dataset is severely biased.

Dataset validation failed due to trait bias. Final linked data not saved.
